In [52]:
import copy
from copyreg import pickle
from pathlib import Path
import warnings
import stock_indicators as sa
import lightning.pytorch as pl
from etfpy import load_etf
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch
from etfpy import ETF, load_etf, get_available_etfs_list
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [11]:
print(funds[0].history(start="1900-01-01", end="2024-01-01").columns)

[DEBUG] yfinance 2024-11-08 23:55:51,164 wrapper:101 - Entering history()
[DEBUG] yfinance 2024-11-08 23:55:51,165 wrapper:101 -  Entering history()
[DEBUG] yfinance 2024-11-08 23:55:51,166 history:161 - ISF.L: Yahoo GET parameters: {'period1': '1900-01-01 00:00:00-00:01', 'period2': '2024-01-01 00:00:00+00:00', 'interval': '1d', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
[DEBUG] yfinance 2024-11-08 23:55:51,177 history:263 - ISF.L: yfinance received OHLC data: 2009-01-02 08:00:00 -> 2023-12-29 08:00:00
[DEBUG] yfinance 2024-11-08 23:55:51,181 history:306 - ISF.L: OHLC after cleaning: 2009-01-02 08:00:00+00:00 -> 2023-12-29 08:00:00+00:00
[DEBUG] yfinance 2024-11-08 23:55:51,198 history:375 - ISF.L: OHLC after combining events: 2009-01-02 00:00:00+00:00 -> 2023-12-29 00:00:00+00:00
[DEBUG] yfinance 2024-11-08 23:55:51,205 history:453 - ISF.L: yfinance returning OHLC: 2009-01-02 00:00:00+00:00 -> 2023-12-29 00:00:00+00:00
[DEBUG] yfinance 2024-11-08 23:55:51,206 wrapp

In [37]:
pos_desc = []
data = {}

In [5]:
porftolio_alloc = pd.read_csv('porftolio_alloc.csv', sep=",")
porftolio_alloc

,ISIN,Position Description,Conservative,Moderate Conservative,Moderate,Moderate Aggressive,Aggressive
0,IE0005042456,ISHARES CORE FTSE 100 UCITS ETF GBP (DIST),-,-,-,-,1.50%
1,IE000T9EOCL3,ISHARES MSCI WORLD SMALL CAP ESG ENHANCED UCIT...,1.00%,1.50%,2.00%,2.50%,3.50%
2,IE00B3ZW0K18,ISHARES S&P 500 EUR HEDGED UCITS ETF (ACC),4.00%,8.50%,11.50%,13.50%,18.00%
3,IE00B52MJY50,ISHARES CORE MSCI PACIFIC EX-JAPAN UCITS ETF U...,-,-,1.00%,1.00%,1.00%
4,IE00B52VJ196,ISHARES MSCI EUROPE SRI UCITS ETF EUR (ACC),-,1.00%,-,-,-
5,IE00BFNM3G45,ISHARES MSCI USA ESG SCREENED UCITS ETF USD (ACC),-,-,-,6.50%,12.50%
6,IE00BHZPJ452,ISHARES MSCI JAPAN ESG ENHANCED UCITS ETF USD ...,1.50%,3.00%,4.00%,6.00%,7.50%
7,IE00BHZPJ783,ISHARES MSCI EUROPE ESG ENHANCED UCITS ETF EUR...,-,-,2.50%,3.00%,7.00%
8,IE00BHZPJ908,ISHARES MSCI USA ESG ENHANCED UCITS ETF USD (ACC),6.00%,12.50%,16.50%,18.00%,18.00%
9,IE00BYYHSM20,ISHARES MSCI EUROPE QUALITY DIVIDEND ESG UCITS...,2.00%,3.50%,3.50%,6.00%,4.50%


In [6]:
chanes_in_pa = pd.read_csv('chanes_in_pa.csv')
chanes_in_pa

,ISIN,Position Description,Conservative,Moderate Conservative,Moderate,Moderate Aggressive,Aggressive
0,IE0005042456,ISHARES CORE FTSE 100 UCITS ETF GBP (DIST),-,-,-,-,-
1,IE000T9EOCL3,ISHARES MSCI WORLD SMALL CAP ESG ENHANCED UCIT...,1.00%,1.50%,2.00%,2.50%,3.50%
2,IE00B1XNHC34,ISHARES GLOBAL CLEAN ENERGY UCITS ETF USD (DIST),-,-,-1.00%,-1.00%,-1.50%
3,IE00B3ZW0K18,ISHARES S&P 500 EUR HEDGED UCITS ETF (ACC),-,-,-,-,-
4,IE00B52MJY50,ISHARES CORE MSCI PACIFIC EX-JAPAN UCITS ETF U...,-,-,-,-,-
5,IE00B52VJ196,ISHARES MSCI EUROPE SRI UCITS ETF EUR (ACC),-,-0.50%,-,-,-
6,IE00BFNM3G45,ISHARES MSCI USA ESG SCREENED UCITS ETF USD (ACC),-,-,-,-,-
7,IE00BHZPJ015,ISHARES MSCI EMU ESG ENHANCED UCITS ETF EUR (ACC),-,-,-,-2.50%,-2.50%
8,IE00BHZPJ452,ISHARES MSCI JAPAN ESG ENHANCED UCITS ETF USD ...,-0.50%,-0.50%,-1.00%,-1.00%,-1.00%
9,IE00BHZPJ783,ISHARES MSCI EUROPE ESG ENHANCED UCITS ETF EUR...,-,-,-,-,-


In [3]:
import os
prices_of_funds = {}
for file in os.listdir("ISINS"):
    if file.endswith(".csv"):
        nu_file = file.replace(".csv","")
        df = pd.read_csv(f"ISINS/{file}")
        prices_of_funds[nu_file] = df
        prices_of_funds[nu_file].index = prices_of_funds[nu_file]["date"]
        del prices_of_funds[nu_file]["date"]
        prices_of_funds[nu_file]["daily_relative"] = prices_of_funds[nu_file]["quote"].pct_change() * 100
        prices_of_funds[nu_file]['continuous_roi'] = (np.log(prices_of_funds[nu_file]["quote"] / prices_of_funds[nu_file]["quote"].shift(1)))
        prices_of_funds[nu_file].fillna(0, inplace=True)

In [56]:
import yfinance as yf
import os
files = os.listdir("ISINS")
files = [file.replace(".csv", "") for file in files]

funds=[]
for fund in files:
    a = (pd.DataFrame(yf.Ticker(fund).history(start="2006-01-01", end="2024-01-01")))
    funds.append(a)
    
for fund in funds:
    if 'Dividends' in fund.columns:
        fund.drop(['Dividends'], axis=1, inplace=True)
    if 'Stock Splits' in fund.columns:
        fund.drop(['Stock Splits'], axis=1, inplace=True)
    if 'Capital Gains' in fund.columns:
        fund.drop(["Capital Gains"], axis=1, inplace=True)
    fund["date"]=fund.index

[DEBUG] yfinance 2024-11-09 00:25:06,083 wrapper:101 - Entering history()
[DEBUG] peewee 2024-11-09 00:25:06,084 execute_sql:3319 - ('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['ISF.L', 1, 0])
[DEBUG] yfinance 2024-11-09 00:25:06,084 wrapper:101 -  Entering history()
[DEBUG] yfinance 2024-11-09 00:25:06,085 history:161 - ISF.L: Yahoo GET parameters: {'period1': '2006-01-01 00:00:00+00:00', 'period2': '2024-01-01 00:00:00+00:00', 'interval': '1d', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
[DEBUG] yfinance 2024-11-09 00:25:06,085 wrapper:101 -   Entering get()
[DEBUG] yfinance 2024-11-09 00:25:06,086 wrapper:101 -    Entering _make_request()
[DEBUG] yfinance 2024-11-09 00:25:06,086 _make_request:346 - url=https://query2.finance.yahoo.com/v8/finance/chart/ISF.L
[DEBUG] yfinance 2024-11-09 00:25:06,087 _make_request:347 - params=frozendict.frozendict({'period1': 1136073600, 'period2': 1704067200, 'interval': '1d', 'incl

In [62]:
print(funds[0]["date"])

Date
2009-01-02 00:00:00+00:00   2009-01-02 00:00:00+00:00
2009-01-05 00:00:00+00:00   2009-01-05 00:00:00+00:00
2009-01-06 00:00:00+00:00   2009-01-06 00:00:00+00:00
2009-01-07 00:00:00+00:00   2009-01-07 00:00:00+00:00
2009-01-08 00:00:00+00:00   2009-01-08 00:00:00+00:00
                                       ...           
2023-12-21 00:00:00+00:00   2023-12-21 00:00:00+00:00
2023-12-22 00:00:00+00:00   2023-12-22 00:00:00+00:00
2023-12-27 00:00:00+00:00   2023-12-27 00:00:00+00:00
2023-12-28 00:00:00+00:00   2023-12-28 00:00:00+00:00
2023-12-29 00:00:00+00:00   2023-12-29 00:00:00+00:00
Name: date, Length: 3787, dtype: datetime64[ns, Europe/London]


In [112]:
import datetime
from stock_indicators import Quote
fund_smass = {}

for idx, fund in enumerate(funds):
    quotes_list = [
        Quote(d,o,h,l,c,v) for d,o,h,l,c,v in zip(fund['date'], fund['Open'], fund['High'], fund['Low'], fund['Close'], fund['Volume'])
    ]
    fund_smass[files[idx]] = sa.indicators.get_sma(quotes_list, 30)
    for s in fund_smass[files[idx]]:
        pass
        #gave up goodnight

In [114]:
# pawel: "to to bym wypierdolil i pomyslal o tym jutro bo to pewnie jakas glupota ale nie wiemy jeszcze
# import time
# for i in range(len(funds)):
#     for sma in smas: 
#         time.sleep(1)
#         
#         tmp=[]
#         for s in sma:
#             tmp.append(s.sma)
#             
#     res[i]=tmp

In [122]:
funds_smass_df = pd.DataFrame(fund_smass)
funds_smass_df.columns = files

ValueError: All arrays must be of the same length

In [110]:
funds_smas.to_csv("fund_smas.csv")

In [80]:
import pickle

with open("sma_results.pkl", "wb") as f:
    pickle.dump(res, f)

In [83]:
with (open("sma_results.pkl", "rb") as f):
    a = pickle.load(f)